# Aluno: Lucas Couto Lima RA: 220696

# Atividade
Utilizar o groq.com para usar a API do Llama 3.1 (8b ou 70b) para fazer análise de sentimentos do IMDB.

Faça uma amostragem balanceada do dataset (sugestão: 10%)

Comparar zero-shot com alguma técnica de engenharia de prompts (CoT, self-consistency, etc.).

In [ ]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset
from collections import Counter

test_dataset = load_dataset("stanfordnlp/imdb", split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
def balance_dataset(dataset, num_samples_per_class=500, max_words=100):
    # Inicialize contadores e listas para armazenar amostras balanceadas
    counter_label = Counter({0: 0, 1: 0})
    balanced_dataset = []

    # Itere sobre o dataset e selecione as amostras balanceadas
    for sample in dataset:
        label = sample["label"]
        text = sample["text"]

        # Conta o número de palavras no texto
        num_words = len(text.split())
        # Verifica se o texto tem no máximo 100 palavras e se ainda precisa de mais amostras dessa classe
        if num_words <= max_words and counter_label[label] < num_samples_per_class:
            balanced_dataset.append(sample)
            counter_label[label] += 1

        # Para o loop se já tiver o número desejado de amostras para ambas as classes
        if counter_label[0] >= num_samples_per_class and counter_label[1] >= num_samples_per_class:
            break

    return balanced_dataset, counter_label

# Balanceando o conjunto de teste
test_dataset, counter_label = balance_dataset(list(test_dataset))

print("Teste balanceado:")
print(f"Número de amostras positivas: {counter_label[1]}")
print(f"Número de amostras negativas: {counter_label[0]}")

Teste balanceado:
Número de amostras positivas: 500
Número de amostras negativas: 500


In [ ]:
test_dataset[:3]

[{'text': '<br /><br />Never ever take a film just for its good looking title.<br /><br />Although it all starts well, the film suffers the same imperfections you see in B-films. Its like at a certain moment the writer does not any more how to end the film, so he ends it in a way nobody suspects it thinking this way he is ingenious.<br /><br />A film to be listed on top of the garbage list.<br /><br />',
  'label': 0},
 {'text': "Widow hires a psychopath as a handyman. Sloppy film noir thriller which doesn't make much of its tension promising set-up. (3/10)",
  'label': 0},
 {'text': 'A sprawling, overambitious, plotless comedy that has no dramatic center. It was probably intended to have an epic vision and a surrealistic flair (at least in some episodes), but the separate stories are never elevated into a meaningful whole, and the laughs are few and far between. Amusing ending, though. (*1/2)',
  'label': 0}]

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [ ]:
from groq import Groq

client = Groq(api_key="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")
completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "system",
            "content": "Classify this movie review as positive or negative. Use this format <positive/negative>"
        },
        {
            "role": "user",
            "content": "Every movie I have PPV'd because Leonard Maltin praised it to the skies has blown chunks! Every single one! When will I ever learn?<br /><br />Evie is a raving Old Bag who thinks nothing of saying she's dying of breast cancer to get her way! Laura is an insufferable Medusa filled with The Holy Spirit (and her hubby's protégé)! Caught between these harpies is Medusa's dumb-as-a-rock boy who has been pressed into weed-pulling servitude by The Old Bag!<br /><br />As I said, when will I ever learn?<br /><br />I was temporarily lifted out of my malaise when The Old Bag stuck her head in a sink, but, unfortunately, she did not die. I was temporarily lifted out of my malaise again when Medusa got mowed down, but, unfortunately, she did not die. It should be a capital offense to torture audiences like this!<br /><br />Without Harry Potter to kick him around, Rupert Grint is just a pair of big blue eyes that practically bulge out of its sockets. Julie Walters's scenery-chewing (especially the scene when she plays God) is even more shameless than her character.<br /><br />At least this Harold bangs some bimbo instead of Maude. For that, I am truly grateful. And if you're reading this Mr. Maltin, you owe me $3.99!"
        }
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,

)

print(completion.choices[0].message.content.strip().lower())

negative


In [ ]:
completion

ChatCompletion(id='chatcmpl-530e445b-a4e1-4cba-86d9-92633a89a1ed', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='negative', role='assistant', function_call=None, tool_calls=None))], created=1727265743, model='llama3-8b-8192', object='chat.completion', system_fingerprint='fp_179b0f92c9', usage=CompletionUsage(completion_tokens=2, prompt_tokens=332, total_tokens=334, completion_time=0.001666667, prompt_time=0.044403482, queue_time=0.0017909539999999974, total_time=0.046070149), x_groq={'id': 'req_01j8mgz8qyfj9trx51fe2q4v6f'})

# One-shot

In [ ]:
def classify_text(text):
    completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "system",
            "content": "Classify this movie review as positive or negative. Use this format: Answer: <positive/negative>"
        },
        {
            "role": "user",
            "content": text
        }
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    )

    #print(chunk_content)
    if "positive" in completion.choices[0].message.content.strip().lower():
        return 1
    else:
        return 0

In [ ]:
import time
from tqdm import tqdm
# Função para calcular a acurácia do dataset
def evaluate_dataset(dataset):
    correct_predictions = 0
    total_samples = len(dataset)

    # Usa tqdm para criar a barra de progresso
    for data in tqdm(dataset):
        text = data['text']
        true_label = data['label']

        # Classifica o texto usando a API
        predicted_label = classify_text(text)

        # Compara a previsão com o rótulo verdadeiro
        if predicted_label == true_label:
            correct_predictions += 1

    # Calcula a acurácia
    accuracy = correct_predictions / total_samples
    return accuracy

# Calcular a acurácia no test_dataset
accuracy = evaluate_dataset(test_dataset)
print(f" Acurácia: {accuracy * 100:.2f}%")

100%|██████████| 1000/1000 [35:33<00:00,  2.13s/it]

 Acurácia: 93.50%


# Few-shot 2 exemplos

In [ ]:
train_dataset = load_dataset("stanfordnlp/imdb", split="train")

In [ ]:
# Balanceando o conjunto de teste
train_dataset, train_counter_label = balance_dataset(list(train_dataset))

print("Teste balanceado:")
print(f"Número de amostras positivas: {train_counter_label[1]}")
print(f"Número de amostras negativas: {train_counter_label[0]}")

Teste balanceado:
Número de amostras positivas: 500
Número de amostras negativas: 500


In [ ]:
import random

# Função para selecionar aleatoriamente um texto com label 0 (negative) e um com label 1 (positive)
def select_random_texts(dataset):
    # Filtra os textos com base nos rótulos
    negative_samples = [sample for sample in dataset if sample['label'] == 0]
    positive_samples = [sample for sample in dataset if sample['label'] == 1]

    # Seleciona aleatoriamente um texto de cada grupo
    negative_sample = random.choice(negative_samples)
    positive_sample = random.choice(positive_samples)

    return negative_sample['text'], positive_sample['text']

# Exemplo de uso com o train_dataset
negative_text, positive_text = select_random_texts(train_dataset)

# Mostrando os textos selecionados
print("Texto negativo:", negative_text)
print("Texto positivo:", positive_text)


Texto negativo: In this day and age of incredible special movie effects, this one was a sore disappointment. The actors seemed stiff and uninspired, as was the dialogue. Westerns are not common fare for Hollywood so much these days, but movies like "Silverado" prove that somebody out there still knows how to make a good one. Considering that, it is hard to conceive that anyone would go to any expense at all in releasing, much less creating such a weak film as this one. If you love and are looking for a good western, keep looking!
Texto positivo: I liked the film. Some of the action scenes were very interesting, tense and well done. I especially liked the opening scene which had a semi truck in it. A very tense action scene that seemed well done.<br /><br />Some of the transitional scenes were filmed in interesting ways such as time lapse photography, unusual colors, or interesting angles. Also the film is funny is several parts. I also liked how the evil guy was portrayed too. I'd give

In [ ]:
#def few_shot(text, negative, positive)
completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "system",
            "content": "Classify this movie review as positive or negative. Use this format <positive/negative>"
        },
        {
            "role": "assistant",
            "content": f"For example, this is a positive review: {positive_text}"
        },
        {
            "role": "assistant",
            "content": f"For example, this is a negative review: {negative_text}"
        },
        {
            "role": "user",
            "content": "Every movie I have PPV'd because Leonard Maltin praised it to the skies has blown chunks! Every single one! When will I ever learn?<br /><br />Evie is a raving Old Bag who thinks nothing of saying she's dying of breast cancer to get her way! Laura is an insufferable Medusa filled with The Holy Spirit (and her hubby's protégé)! Caught between these harpies is Medusa's dumb-as-a-rock boy who has been pressed into weed-pulling servitude by The Old Bag!<br /><br />As I said, when will I ever learn?<br /><br />I was temporarily lifted out of my malaise when The Old Bag stuck her head in a sink, but, unfortunately, she did not die. I was temporarily lifted out of my malaise again when Medusa got mowed down, but, unfortunately, she did not die. It should be a capital offense to torture audiences like this!<br /><br />Without Harry Potter to kick him around, Rupert Grint is just a pair of big blue eyes that practically bulge out of its sockets. Julie Walters's scenery-chewing (especially the scene when she plays God) is even more shameless than her character.<br /><br />At least this Harold bangs some bimbo instead of Maude. For that, I am truly grateful. And if you're reading this Mr. Maltin, you owe me $3.99!"
        }
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,

)

print(completion.choices[0].message.content.strip().lower())

negative


In [ ]:
completion

ChatCompletion(id='chatcmpl-ca007f17-a53f-4025-a561-51a903a634f0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='negative', role='assistant', function_call=None, tool_calls=None))], created=1727216934, model='llama3-8b-8192', object='chat.completion', system_fingerprint='fp_873a560973', usage=CompletionUsage(completion_tokens=2, prompt_tokens=434, total_tokens=436, completion_time=0.001666667, prompt_time=0.079890232, queue_time=0.0020707269999999944, total_time=0.081556899), x_groq={'id': 'req_01j8k2dqt3fmva3mkjpgtc7q1a'})

In [ ]:
def few_shot(text, negative, positive):
    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {
                "role": "system",
                "content": "Classify this movie review as positive or negative. Use this format <positive/negative>"
            },
            {
                "role": "assistant",
                "content": f"For example, this is a positive review: {positive}"
            },
            {
                "role": "assistant",
                "content": f"For example, this is a negative review: {negative}"
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,

    )
    if "positive" in completion.choices[0].message.content.strip().lower():
        return 1
    else:
        return 0

In [ ]:
# Função para calcular a acurácia do dataset
def evaluate_dataset_fewshot(dataset):
    correct_predictions = 0
    total_samples = len(dataset)

    # Usa tqdm para criar a barra de progresso
    for data in tqdm(dataset):
        text = data['text']
        true_label = data['label']
        negative_text, positive_text = select_random_texts(train_dataset)
        # Classifica o texto usando a API
        predicted_label = few_shot(text, negative_text, positive_text)

        # Compara a previsão com o rótulo verdadeiro
        if predicted_label == true_label:
            correct_predictions += 1

    # Calcula a acurácia
    accuracy = correct_predictions / total_samples
    return accuracy

# Calcular a acurácia no test_dataset
accuracy = evaluate_dataset_fewshot(test_dataset)
print(f" Acurácia: {accuracy * 100:.2f}%")

100%|██████████| 1000/1000 [36:25<00:00,  2.19s/it]

 Acurácia: 93.30%


# CoT

In [ ]:
def extract_answer(text):
    # Divide o texto na palavra "answer:" e pega a segunda parte (após "answer:")
    answer_part = text.split("Answer:")[-1].strip()
    return answer_part

In [ ]:
client = Groq(api_key="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")
def CoT(text):
    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {
                "role": "system",
                "content": "Classify this movie review as positive or negative. Use this format: Explanation: <> Answer: <positive/negative>"
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,

    )

    if "positive" in extract_answer(completion.choices[0].message.content.strip().lower()):
        return 1
    else:
        return 0

In [ ]:
from tqdm import tqdm
# Função para calcular a acurácia do dataset
def evaluate_dataset(dataset):
    correct_predictions = 0
    total_samples = len(dataset)

    # Usa tqdm para criar a barra de progresso
    for data in tqdm(dataset):
        text = data['text']
        true_label = data['label']

        # Classifica o texto usando a API
        predicted_label = CoT(text)

        # Compara a previsão com o rótulo verdadeiro
        if predicted_label == true_label:
            correct_predictions += 1

    # Calcula a acurácia
    accuracy = correct_predictions / total_samples
    return accuracy

# Calcular a acurácia no test_dataset
accuracy = evaluate_dataset(test_dataset)
print(f" Acurácia: {accuracy * 100:.2f}%")

100%|██████████| 1000/1000 [39:15<00:00,  2.36s/it]

 Acurácia: 88.50%


# Few-shot 4 exemplos

In [ ]:
client = Groq(api_key="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")
def few_shot_4(text, negative, positive, negative_2, positive_2):
    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {
                "role": "system",
                "content": "Classify this movie review as positive or negative. Use this format <positive/negative>"
            },
            {
                "role": "assistant",
                "content": f"For example, this is a positive review: {positive}"
            },
            {
                "role": "assistant",
                "content": f"For example, this is a negative review: {negative}"
            },
            {
                "role": "assistant",
                "content": f"For example, this is a positive review: {positive_2}"
            },
            {
                "role": "assistant",
                "content": f"For example, this is a negative review: {negative_2}"
            },
            {
                "role": "user",
                "content": text
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,

    )
    if "positive" in completion.choices[0].message.content.strip().lower():
        return 1
    else:
        return 0

In [ ]:
# Função para calcular a acurácia do dataset
def evaluate_dataset_fewshot_4(dataset):
    correct_predictions = 0
    total_samples = len(dataset)

    # Usa tqdm para criar a barra de progresso
    for data in tqdm(dataset):
        text = data['text']
        true_label = data['label']
        negative_text, positive_text = select_random_texts(train_dataset)
        negative_text_2, positive_text_2 = select_random_texts(train_dataset)
        # Classifica o texto usando a API
        predicted_label = few_shot_4(text, negative_text, positive_text, negative_text_2, positive_text_2)

        # Compara a previsão com o rótulo verdadeiro
        if predicted_label == true_label:
            correct_predictions += 1

    # Calcula a acurácia
    accuracy = correct_predictions / total_samples
    return accuracy

# Calcular a acurácia no test_dataset
accuracy = evaluate_dataset_fewshot_4(test_dataset)
print(f" Acurácia: {accuracy * 100:.2f}%")

 92%|█████████▏| 921/1000 [37:17<03:11,  2.43s/it]


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j8mqdnxxe9qvcs4kawbgkw91` on : Limit 500000, Used 499968, Requested 471. Please try again in 1m15.6922s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}